下面，我们基于gnt-H的10层模型，应用上左右随机30度以内的旋转，再加上gamma随机校正的训练数据，进行训练。以观察精度增长情况。

这个应该很耗时。但是，希望在4万步时，精度能超过83%。（8层在4万步时，精度达到了84%）


In [1]:
import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.exposure import adjust_gamma
from skimage.transform import rotate

IMAGE_SIZE = 64
FULL_CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def getRndGamma():
    rnd1 = np.random.random_sample()
    rnd2 = np.random.random_sample()+0.001
    rnd3 = np.random.random_sample()*100

    if (rnd1 > 0.5):
        return rnd3
    else:
        return rnd2
    
def input(dataset):
    return dataset.images, dataset.labels

class DataSetLoader:
    def __init__(self, data_dir):
        # Set CHARSET_SIZE to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        
        # add random rotate
        image_rotated = [rotate(image, np.random.random_sample()*60-30) for image in image_resized]

        # add random gamma adjust
        image_gamma_adjusted = [adjust_gamma(image, getRndGamma()) for image in image_rotated]

        self.images = [rgb2gray(item) for item in image_gamma_adjusted]
        
        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

# Build 8 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns, # The input features to our model
    hidden_units=[4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], # 8 hidden layers
    n_classes=CHARSET_SIZE, # survived or not {1, 0}
    model_dir="../dfs/checkpoint/dnn10_model_b", # Path to where checkpoints etc are stored
    optimizer=tf.train.RMSPropOptimizer(
        learning_rate=0.00001),
    dropout=0.1)

for i in range(10):
    train_data = DataSetLoader(data_dir='../data/train_/')
    print ('Train data loaded for the %d times.'%(i+1))
    # Define the training inputs
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": input(train_data)[0]},
        y=input(train_data)[1],
        num_epochs=None,
        batch_size=50,
        shuffle=True
    )

    classifier.train(input_fn=train_input_fn, steps=2000)

print ('Train done, begin to test ...')
test_data = DataSetLoader(data_dir='../data/test_/')
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../dfs/checkpoint/dnn10_model_b', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
Now processing path:  ../data/train_/00037
Train data loaded for the 1 times.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:loss = 181.591, step = 1
INFO:tensorflow:global_step/sec: 0.488055
INFO:tensorflow:loss = 180.581, step = 101 (204.901 sec)
INFO:tensorflow:global_step/sec: 0.530987
INFO:tensorflow:loss = 179.667, step = 201 (188.325 sec)
INFO:tensorflow:Saving checkpoints for 285 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:global_step/sec: 0.413516
INFO:tensorflow:loss = 175.888, step = 301 (241.832 sec)
INF

INFO:tensorflow:global_step/sec: 0.415736
INFO:tensorflow:loss = 54.469, step = 6001 (240.542 sec)
INFO:tensorflow:global_step/sec: 0.491759
INFO:tensorflow:loss = 44.5088, step = 6101 (203.354 sec)
INFO:tensorflow:global_step/sec: 0.549115
INFO:tensorflow:loss = 26.2295, step = 6201 (182.114 sec)
INFO:tensorflow:Saving checkpoints for 6268 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:global_step/sec: 0.408476
INFO:tensorflow:loss = 37.1206, step = 6301 (244.805 sec)
INFO:tensorflow:global_step/sec: 0.492327
INFO:tensorflow:loss = 26.4244, step = 6401 (203.115 sec)
INFO:tensorflow:global_step/sec: 0.544708
INFO:tensorflow:loss = 29.2676, step = 6501 (183.590 sec)
INFO:tensorflow:Saving checkpoints for 6552 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:global_step/sec: 0.408277
INFO:tensorflow:loss = 32.3923, step = 6601 (244.929 sec)
INFO:tensorflow:global_step/sec: 0.488974
INFO:tensorflow:loss = 41.2293, step = 6701 (204.514 sec)
INFO:tensorflow

INFO:tensorflow:loss = 28.909, step = 12801 (131.549 sec)
INFO:tensorflow:global_step/sec: 0.805344
INFO:tensorflow:loss = 38.9052, step = 12901 (124.171 sec)
INFO:tensorflow:Saving checkpoints for 12911 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:global_step/sec: 0.863697
INFO:tensorflow:loss = 38.6052, step = 13001 (115.781 sec)
INFO:tensorflow:global_step/sec: 1.03207
INFO:tensorflow:loss = 38.0433, step = 13101 (96.893 sec)
INFO:tensorflow:global_step/sec: 1.03495
INFO:tensorflow:loss = 43.0396, step = 13201 (96.622 sec)
INFO:tensorflow:global_step/sec: 1.03563
INFO:tensorflow:loss = 30.5497, step = 13301 (96.560 sec)
INFO:tensorflow:global_step/sec: 1.03037
INFO:tensorflow:loss = 30.4961, step = 13401 (97.052 sec)
INFO:tensorflow:global_step/sec: 1.03523
INFO:tensorflow:loss = 32.5001, step = 13501 (96.597 sec)
INFO:tensorflow:Saving checkpoints for 13512 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:global_step/sec: 0.868582
INFO:tensorflow

INFO:tensorflow:global_step/sec: 0.874434
INFO:tensorflow:loss = 16.3706, step = 19901 (114.359 sec)
INFO:tensorflow:Saving checkpoints for 20000 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:Loss for final step: 20.9473.
Now processing path:  ../data/train_/00037
Train data loaded for the 3 times.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/dnn10_model_b/model.ckpt-20000
INFO:tensorflow:Saving checkpoints for 20001 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:loss = 81.1521, step = 20001
INFO:tensorflow:global_step/sec: 1.06724
INFO:tensorflow:loss = 47.2308, step = 20101 (93.701 sec)
INFO:tensorflow:global_step/sec: 1.0363
INFO:tensorflow:loss = 51.3604, step = 20201 (96.498 sec)
INFO:tensorflow:global_step/sec: 1.03224
INFO:tensorflow:loss = 35.1522, step = 20301 (96.877 sec)
INFO:tensorflow:global_step/sec: 1.0327
INFO:tensorflow:loss = 20.6792, step = 20401 (96.833 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 0.861498
INFO:tensorflow:loss = 17.9069, step = 26701 (116.077 sec)
INFO:tensorflow:Saving checkpoints for 26757 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:global_step/sec: 0.747458
INFO:tensorflow:loss = 17.0969, step = 26801 (133.787 sec)
INFO:tensorflow:global_step/sec: 0.855478
INFO:tensorflow:loss = 21.8202, step = 26901 (116.894 sec)
INFO:tensorflow:global_step/sec: 0.861587
INFO:tensorflow:loss = 13.0403, step = 27001 (116.065 sec)
INFO:tensorflow:global_step/sec: 0.85523
INFO:tensorflow:loss = 37.188, step = 27101 (116.928 sec)
INFO:tensorflow:global_step/sec: 0.86005
INFO:tensorflow:loss = 4.56447, step = 27201 (116.272 sec)
INFO:tensorflow:Saving checkpoints for 27257 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:global_step/sec: 0.743164
INFO:tensorflow:loss = 23.9663, step = 27301 (134.560 sec)
INFO:tensorflow:global_step/sec: 0.861749
INFO:tensorflow:loss = 31.9824, step = 27401 (116.042 sec)
INFO:te

INFO:tensorflow:loss = 18.566, step = 33201 (96.660 sec)
INFO:tensorflow:global_step/sec: 1.03988
INFO:tensorflow:loss = 23.4909, step = 33301 (96.165 sec)
INFO:tensorflow:global_step/sec: 1.03436
INFO:tensorflow:loss = 15.3061, step = 33401 (96.678 sec)
INFO:tensorflow:global_step/sec: 1.03698
INFO:tensorflow:loss = 18.4845, step = 33501 (96.434 sec)
INFO:tensorflow:global_step/sec: 1.0374
INFO:tensorflow:loss = 19.1817, step = 33601 (96.395 sec)
INFO:tensorflow:Saving checkpoints for 33698 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:global_step/sec: 0.874106
INFO:tensorflow:loss = 40.3502, step = 33701 (114.403 sec)
INFO:tensorflow:global_step/sec: 1.04013
INFO:tensorflow:loss = 14.2003, step = 33801 (96.142 sec)
INFO:tensorflow:global_step/sec: 1.04081
INFO:tensorflow:loss = 16.9591, step = 33901 (96.080 sec)
INFO:tensorflow:global_step/sec: 1.03744
INFO:tensorflow:loss = 35.6904, step = 34001 (96.391 sec)
INFO:tensorflow:global_step/sec: 1.04023
INFO:tensorflow

INFO:tensorflow:global_step/sec: 1.20909
INFO:tensorflow:loss = 42.5154, step = 40101 (82.704 sec)
INFO:tensorflow:global_step/sec: 1.25626
INFO:tensorflow:loss = 35.9109, step = 40201 (79.603 sec)
INFO:tensorflow:global_step/sec: 1.26258
INFO:tensorflow:loss = 31.7463, step = 40301 (79.202 sec)
INFO:tensorflow:global_step/sec: 1.27344
INFO:tensorflow:loss = 47.0521, step = 40401 (78.526 sec)
INFO:tensorflow:global_step/sec: 1.27563
INFO:tensorflow:loss = 22.978, step = 40501 (78.392 sec)
INFO:tensorflow:global_step/sec: 1.27726
INFO:tensorflow:loss = 25.2481, step = 40601 (78.293 sec)
INFO:tensorflow:global_step/sec: 1.27851
INFO:tensorflow:loss = 30.3149, step = 40701 (78.216 sec)
INFO:tensorflow:Saving checkpoints for 40736 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:global_step/sec: 1.03537
INFO:tensorflow:loss = 39.5242, step = 40801 (96.584 sec)
INFO:tensorflow:global_step/sec: 1.28132
INFO:tensorflow:loss = 20.8354, step = 40901 (78.044 sec)
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 1.0248
INFO:tensorflow:loss = 9.45741, step = 47501 (97.579 sec)
INFO:tensorflow:global_step/sec: 1.27072
INFO:tensorflow:loss = 13.2045, step = 47601 (78.696 sec)
INFO:tensorflow:global_step/sec: 1.27149
INFO:tensorflow:loss = 8.10369, step = 47701 (78.648 sec)
INFO:tensorflow:global_step/sec: 1.27132
INFO:tensorflow:loss = 11.5456, step = 47801 (78.659 sec)
INFO:tensorflow:global_step/sec: 1.27395
INFO:tensorflow:loss = 12.7415, step = 47901 (78.496 sec)
INFO:tensorflow:global_step/sec: 1.26898
INFO:tensorflow:loss = 36.4556, step = 48001 (78.803 sec)
INFO:tensorflow:global_step/sec: 1.26815
INFO:tensorflow:loss = 20.7592, step = 48101 (78.855 sec)
INFO:tensorflow:Saving checkpoints for 48175 into ../dfs/checkpoint/dnn10_model_b/model.ckpt.
INFO:tensorflow:global_step/sec: 1.02888
INFO:tensorflow:loss = 24.1202, step = 48201 (97.193 sec)
INFO:tensorflow:global_step/sec: 1.26602
INFO:tensorflow:loss = 32.6088, step = 48301 (78.988 sec)
INFO:tensorflow:

下面是单独测试：

In [1]:
# from: https://www.kaggle.com/jeffcarp/example-save-and-load-a-tensorflow-model

import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

from tensorflow.contrib.learn.python.learn.estimators import run_config
from tensorflow.contrib.training.python.training import hparam

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

class DataSetLoader:
    def __init__(self, data_dir):
        # Set CHARSET_SIZE to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        
        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

def make_estimator(model_dir):
    config = run_config.RunConfig(model_dir=model_dir)

    return tf.estimator.DNNClassifier (
        config=config,
        feature_columns=feature_columns, # The input features to our model
    hidden_units=[4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], # 8 hidden layers
    n_classes=CHARSET_SIZE, # survived or not {1, 0}
    optimizer=tf.train.RMSPropOptimizer(
        learning_rate=0.00001),
    dropout=0.1)
    

MODEL_DIR = "../dfs/checkpoint/dnn10_model_b"
model_from_checkpoint = make_estimator(MODEL_DIR)

test_data = DataSetLoader(data_dir='../data/test_/')
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy
accuracy_score = model_from_checkpoint.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))


INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fce07fde630>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '../dfs/checkpoint/dnn10_model_b'}
Now processing path:  ../data/test_/00037
INFO:tensorflow:Starting evaluation at 2018-04-24-00:46:55
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/dnn10_model_b/model.ckpt-50000
INFO:tensorflow:Finished evaluation at 2018-04-24-00:47:03
INFO:tensorflow:Saving dict for global step 50000: accuracy = 0.795475, average_loss = 1.8802

目前看来，精度不高：

INFO:tensorflow:Starting evaluation at 2018-04-24-00:46:55
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/dnn10_model_b/model.ckpt-50000
INFO:tensorflow:Finished evaluation at 2018-04-24-00:47:03
INFO:tensorflow:Saving dict for global step 50000: accuracy = 0.795475, average_loss = 1.88029, global_step = 50000, loss = 230.858

Test Accuracy: 79.547513%

5万步，用时约20小时。
